# Fine-tuning BART model for a summarization task using CNN_dailymail

## Loading the dataset

In [2]:
# Datasets version used was 1.10.2
from datasets import load_dataset, load_metric

In [25]:
raw_datasets = load_dataset("cnn_dailymail", "3.0.0")
metric = load_metric("rouge")

Reusing dataset cnn_dailymail (/home/josmar/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/3cb851bf7cf5826e45d49db2863f627cba583cbc32342df7349dfe6c38060234)


In [ ]:
model_checkpoint = "facebook/bart-base"

In [7]:
print(type(raw_datasets))
print(raw_datasets.keys())
print(raw_datasets['test'])

<class 'datasets.dataset_dict.DatasetDict'>
dict_keys(['train', 'validation', 'test'])
Dataset({
    features: ['article', 'highlights', 'id'],
    num_rows: 11490
})


## Preprocessing the data

In [10]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [11]:
max_input_length = 512
max_target_length = 128

def preprocess_function(examples):
    inputs = [doc for doc in examples["article"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["highlights"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [12]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

100%|██████████| 12/12 [00:03<00:00,  3.02ba/s]


## Fine-tuning the model

In [13]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [14]:
batch_size = 4
args = Seq2SeqTrainingArguments(
    "BART_Finetuned_CNN_dailymail",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    gradient_accumulation_steps=2,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
)

In [15]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [16]:
import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

In [17]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Using amp fp16 backend


In [52]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/josmar/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [19]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: article, id, highlights.
***** Running training *****
  Num examples = 287113
  Num Epochs = 1
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 2
  Total optimization steps = 35889
  0%|          | 0/35889 [00:00<?, ?it/s]/home/josmar/.conda/envs/saturday/lib/python3.6/site-packages/transformers/trainer.py:1299: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  args.max_grad_norm,
  1%|▏         | 500/35889 [02:18<2:45:40,  3.56it/s]Saving model checkpoint to test-dialogue-summarization/checkpo

{'loss': 2.4927, 'learning_rate': 1.9722477639388116e-05, 'epoch': 0.01}


Model weights saved in test-dialogue-summarization/checkpoint-500/pytorch_model.bin
tokenizer config file saved in test-dialogue-summarization/checkpoint-500/tokenizer_config.json
Special tokens file saved in test-dialogue-summarization/checkpoint-500/special_tokens_map.json
Deleting older checkpoint [test-dialogue-summarization/checkpoint-1000] due to args.save_total_limit
  3%|▎         | 1000/35889 [04:39<2:35:04,  3.75it/s]Saving model checkpoint to test-dialogue-summarization/checkpoint-1000
Configuration saved in test-dialogue-summarization/checkpoint-1000/config.json


{'loss': 2.2764, 'learning_rate': 1.944384073114325e-05, 'epoch': 0.03}


Model weights saved in test-dialogue-summarization/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in test-dialogue-summarization/checkpoint-1000/tokenizer_config.json
Special tokens file saved in test-dialogue-summarization/checkpoint-1000/special_tokens_map.json
Deleting older checkpoint [test-dialogue-summarization/checkpoint-1500] due to args.save_total_limit
  4%|▍         | 1500/35889 [06:58<2:38:01,  3.63it/s]Saving model checkpoint to test-dialogue-summarization/checkpoint-1500
Configuration saved in test-dialogue-summarization/checkpoint-1500/config.json


...

Model weights saved in test-dialogue-summarization/checkpoint-35000/pytorch_model.bin
tokenizer config file saved in test-dialogue-summarization/checkpoint-35000/tokenizer_config.json
Special tokens file saved in test-dialogue-summarization/checkpoint-35000/special_tokens_map.json
Deleting older checkpoint [test-dialogue-summarization/checkpoint-34000] due to args.save_total_limit
 99%|█████████▉| 35500/35889 [2:33:14<01:37,  3.99it/s]Saving model checkpoint to test-dialogue-summarization/checkpoint-35500
Configuration saved in test-dialogue-summarization/checkpoint-35500/config.json


{'loss': 1.955, 'learning_rate': 2.2513862186185183e-07, 'epoch': 0.99}


Model weights saved in test-dialogue-summarization/checkpoint-35500/pytorch_model.bin
tokenizer config file saved in test-dialogue-summarization/checkpoint-35500/tokenizer_config.json
Special tokens file saved in test-dialogue-summarization/checkpoint-35500/special_tokens_map.json
Deleting older checkpoint [test-dialogue-summarization/checkpoint-34500] due to args.save_total_limit
100%|██████████| 35889/35889 [2:34:57<00:00,  3.72it/s]The following columns in the evaluation set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: article, id, highlights.
***** Running Evaluation *****
  Num examples = 13368
  Batch size = 4
/home/josmar/.conda/envs/saturday/lib/python3.6/site-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the curr

{'eval_loss': 1.7213754653930664, 'eval_rouge1': 24.3468, 'eval_rouge2': 12.1644, 'eval_rougeL': 20.265, 'eval_rougeLsum': 22.9442, 'eval_gen_len': 20.0, 'eval_runtime': 832.3313, 'eval_samples_per_second': 16.061, 'eval_steps_per_second': 4.015, 'epoch': 1.0}
{'train_runtime': 10129.9819, 'train_samples_per_second': 28.343, 'train_steps_per_second': 3.543, 'train_loss': 2.034251923234117, 'epoch': 1.0}


TrainOutput(global_step=35889, training_loss=2.034251923234117, metrics={'train_runtime': 10129.9819, 'train_samples_per_second': 28.343, 'train_steps_per_second': 3.543, 'train_loss': 2.034251923234117, 'epoch': 1.0})

In [20]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: article, id, highlights.
***** Running Evaluation *****
  Num examples = 13368
  Batch size = 4
100%|██████████| 3342/3342 [13:50<00:00,  4.02it/s]


{'eval_loss': 1.7213754653930664,
 'eval_rouge1': 24.3468,
 'eval_rouge2': 12.1644,
 'eval_rougeL': 20.265,
 'eval_rougeLsum': 22.9442,
 'eval_gen_len': 20.0,
 'eval_runtime': 830.6268,
 'eval_samples_per_second': 16.094,
 'eval_steps_per_second': 4.023,
 'epoch': 1.0}

## Testing the trained model

When the model finished training it was uploaded to Hugging Face to the repository [Josmar/BART_Finetuned_CNN_dailymail](https://huggingface.co/Josmar/BART_Finetuned_CNN_dailymail).
After this, we load the model and run a summarization task.

In [47]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
  
tokenizer_pre = AutoTokenizer.from_pretrained("Josmar/BART_Finetuned_CNN_dailymail")
model_pre = AutoModelForSeq2SeqLM.from_pretrained("Josmar/BART_Finetuned_CNN_dailymail")

https://huggingface.co/Josmar/BART_Finetuned_CNN_dailymail/resolve/main/tokenizer_config.json not found in cache or force_download set to True, downloading to /home/josmar/.cache/huggingface/transformers/tmp8xyhm0e5
Downloading: 100%|██████████| 331/331 [00:00<00:00, 285kB/s]
storing https://huggingface.co/Josmar/BART_Finetuned_CNN_dailymail/resolve/main/tokenizer_config.json in cache at /home/josmar/.cache/huggingface/transformers/e2d15ddc64c4f635d1b8427f8f636a7b01025f40b5acc95aae6cf1ee712f9820.cdcc2d8e582b0301628801f12a68c320bca89103699c95041d4e99522a499c1d
creating metadata file for /home/josmar/.cache/huggingface/transformers/e2d15ddc64c4f635d1b8427f8f636a7b01025f40b5acc95aae6cf1ee712f9820.cdcc2d8e582b0301628801f12a68c320bca89103699c95041d4e99522a499c1d
https://huggingface.co/Josmar/BART_Finetuned_CNN_dailymail/resolve/main/vocab.json not found in cache or force_download set to True, downloading to /home/josmar/.cache/huggingface/transformers/tmpx8gnpp1o
Downloading: 100%|█████████

In [1]:
from transformers import pipeline

dialog_summarizer = pipeline("summarization", model=model_pre, tokenizer=tokenizer_pre)

NameError: name 'model_pre' is not defined

In [49]:
text = raw_datasets["test"][100]['article']
text

'(CNN)It\'s obvious that Tom Brady\'s love for his wife, model Gisele Bundchen, will never go out of fashion. Bundchen walked the runway for the last time Wednesday, and the New England Patriots quarterback wasn\'t just there to support her in person, he expressed his emotions to the world on Facebook. "Congratulations Love of my Life," Brady wrote. "You inspire me every day to be a better person. I am so proud of you and everything you have accomplished on the runway. I have never met someone with more of a will to succeed and determination to overcome any obstacle in the way. You never cease to amaze me. Nobody loves life more than you and your beauty runs much deeper than what the eye can see. I can\'t wait to see what\'s next. I love you." He followed the text with two hashtags, #GOAT ("greatest of all time") and #thebestisyettocome. Bundchen, 34, announced her retirement from the catwalk last weekend. "I am grateful that at 14, I was given the opportunity to start this journey. To

In [50]:
result = dialog_summarizer(text, min_length=10, max_length=40)

In [51]:
result[0]['summary_text']

'Supermodel Gisele Bundchen walked the runway for the last time Wednesday .\nShe announced her retirement from the catwalk last weekend .\nBrady and Bundchen have been married'

In [31]:
trainer.evaluate(eval_dataset=tokenized_datasets["test"])

The following columns in the evaluation set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: article, id, highlights.
***** Running Evaluation *****
  Num examples = 11490
  Batch size = 4
100%|██████████| 2873/2873 [12:08<00:00,  3.94it/s]


{'eval_loss': 1.733933687210083,
 'eval_rouge1': 24.6272,
 'eval_rouge2': 12.1951,
 'eval_rougeL': 20.4885,
 'eval_rougeLsum': 23.1474,
 'eval_gen_len': 20.0,
 'eval_runtime': 728.9095,
 'eval_samples_per_second': 15.763,
 'eval_steps_per_second': 3.942,
 'epoch': 1.0}